In [ ]:
import cv2
from ultralytics import YOLO
from collections import defaultdict
import numpy as np
import math
import timeit
import os


# Load the YOLOv8 model
model = YOLO(r'C:\...\runs\detect\train11\weights\best.engine')

# Open the video file
video_path = r'C:\...\Test\Videos\Clock14.avi'
cap = cv2.VideoCapture(video_path)

goalBlack=0
goalWhite=0
measurementActive=False
counterNoDetections=0
xFrameLeftUp=0
yFrameLeftUp=0
xFrameRightDown=0
yFrameRightDown=0
coordinates=[]
ghostBall=False

# goal-area left side
xLeftArea=105
yUpperLeftArea=150
yDownLeftArea=310
# goal-area right side
xRightArea=665
yUpperRightArea=195
yDownRightArea=320

# time measurement
accumulatedTime=0
frames=0

# save detected frames
#directory = r'C:\...\TrainKicker\SavedPictures'
#os.chdir(directory)


# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    #print(frame)

    if success:
        # Run YOLOv8 tracking on the frame
        start = timeit.default_timer()
        results = model.track(frame, conf=0.6, persist=True)
        accumulatedTime += (timeit.default_timer()-start)
        frames+=1
        
        #result contains all detected Objects
        result = results[0]
        
        if len(result.boxes)==0:
            if measurementActive:
                counterNoDetections+=1
                
            if counterNoDetections==60:
                measurementActive=False
                counterNoDetections=0
                
                if xFrameLeftUp<=xLeftArea and (yFrameLeftUp>=yUpperLeftArea and yFrameRightDown<=yDownLeftArea):
                    goalWhite+=1
                    coordinates.append(xFrameLeftUp)
                    coordinates.append(yFrameLeftUp)
                    coordinates.append(xFrameRightDown)
                    coordinates.append(yFrameRightDown)
                    coordinates.append(-1)
                    
                    xFrameLeftUp=0
                    yFrameLeftUp=0
                    xFrameRightDown=0
                    yFrameRightDown=0
                    
                elif xFrameRightDown>=xRightArea and (yFrameLeftUp>=yUpperRightArea and yFrameRightDown<=yDownRightArea):
                    goalBlack+=1
                    coordinates.append(xFrameLeftUp)
                    coordinates.append(yFrameLeftUp)
                    coordinates.append(xFrameRightDown)
                    coordinates.append(yFrameRightDown)
                    coordinates.append(-1)
                    
                    xFrameLeftUp=0
                    yFrameLeftUp=0
                    xFrameRightDown=0
                    yFrameRightDown=0
            
        elif len(result.boxes)==1:
            
            box = result.boxes[0]
            cords = box.xyxy[0].tolist()
            cords = [round(x) for x in cords]
            ghostBall = False
            distance = math.sqrt(pow((xFrameLeftUp-cords[0]), 2)+pow(yFrameLeftUp-cords[1], 2))
            
            # check if possible ghostball=> case 1: no ball in game, but ghostball in goal triggers measurement and No Detection Counting
            if cords[0]<=xLeftArea and (cords[1]>=yUpperLeftArea and cords[3]<=yDownLeftArea) and xFrameLeftUp==0 and yFrameLeftUp==0 and xFrameRightDown==0 and yFrameRightDown==0:
                ghostBall=True
            elif cords[2]>=xRightArea and (cords[1]>=yUpperRightArea and cords[3]<=yDownRightArea) and xFrameLeftUp==0 and yFrameLeftUp==0 and xFrameRightDown==0 and yFrameRightDown==0:
                ghostBall=True
            # case 2: ball in goal and ghostball in goal interrupts No Detections Counting
            elif xFrameLeftUp<=xLeftArea and (yFrameLeftUp>=yUpperLeftArea and yFrameRightDown<=yDownLeftArea) and cords[0]<=xLeftArea and (cords[1]>=yUpperLeftArea and cords[3]<=yDownLeftArea):
                ghostBall=True
            elif xFrameRightDown>=xRightArea and (yFrameLeftUp>=yUpperRightArea and yFrameRightDown<=yDownRightArea) and cords[2]>=xRightArea and (cords[1]>=yUpperRightArea and cords[3]<=yDownRightArea):
                ghostBall=True
            # case 3: ball in goal and ghostball outside the goal-area interrupts No Detections Counting - Problem mit den weißen Abwehrspielern bzw. Torhüter
            elif xFrameLeftUp<=xLeftArea and (yFrameLeftUp>=yUpperLeftArea and yFrameRightDown<=yDownLeftArea) and distance>=550:
                ghostBall=True
            elif xFrameRightDown>=xRightArea and (yFrameLeftUp>=yUpperRightArea and yFrameRightDown<=yDownRightArea) and distance>=550:
                ghostBall=True
            
            if ghostBall:
                if measurementActive:
                    counterNoDetections+=1
                
                if counterNoDetections==60:
                    measurementActive=False
                    counterNoDetections=0
                
                    if xFrameLeftUp<=xLeftArea and (yFrameLeftUp>=yUpperLeftArea and yFrameRightDown<=yDownLeftArea):
                        goalWhite+=1
                        coordinates.append(xFrameLeftUp)
                        coordinates.append(yFrameLeftUp)
                        coordinates.append(xFrameRightDown)
                        coordinates.append(yFrameRightDown)
                        coordinates.append(-1)
                        
                        xFrameLeftUp=0
                        yFrameLeftUp=0
                        xFrameRightDown=0
                        yFrameRightDown=0
                    
                    elif xFrameRightDown>=xRightArea and (yFrameLeftUp>=yUpperRightArea and yFrameRightDown<=yDownRightArea):
                        goalBlack+=1
                        coordinates.append(xFrameLeftUp)
                        coordinates.append(yFrameLeftUp)
                        coordinates.append(xFrameRightDown)
                        coordinates.append(yFrameRightDown)
                        coordinates.append(-1)
                    
                        xFrameLeftUp=0
                        yFrameLeftUp=0
                        xFrameRightDown=0
                        yFrameRightDown=0
                    
            else:
          
                    # activate measurement if current ball-position in goal area
                    if (cords[0]<=xLeftArea and (cords[1]>=yUpperLeftArea and cords[3]<=yDownLeftArea)) or (cords[2]>=xRightArea and (cords[1]>=yUpperRightArea and cords[3]<=yDownRightArea)):
                        measurementActive=True    
                    elif (cords[0]>xLeftArea and (cords[1]<yUpperLeftArea and cords[3]>yDownLeftArea)) or (cords[2]<xRightArea and (cords[1]<yUpperRightArea and cords[3]>yDownRightArea)):
                        measurementActive=False

                    # set to 0 everytime otherwise goals would be counted during the game (ball hidden/covered by figures, bars, ...)                
                    counterNoDetections=0
            
                    xFrameLeftUp=cords[0]
                    yFrameLeftUp=cords[1]
                    xFrameRightDown=cords[2]
                    yFrameRightDown=cords[3]
                    
        

                
        annotated_frame = results[0].plot()
        # Display the annotated frame
        #(image, 'OpenCV', org, font,fontScale, color, thickness, cv2.LINE_AA) 
        annotated_frame = cv2.putText(annotated_frame, f"{goalBlack} : {goalWhite}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        
        #filename = str(frames) + '_saved_frame.png'
        #cv2.imwrite(filename, annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

print(coordinates)
print(accumulatedTime/frames)
